# Synchronize Annif corpus files against latest YSO

This notebook can be used to update Annif corpus files that have been created using an older version of YSO. It will read the old corpus files, look up YSO URIs in the new YSO version, update their labels in case they have changed, change deprecated URIs to their successors and remove any URIs that are not in the new YSO (should not happen but...).

Requirements:

* jupyter (obviously)
* rdflib

In [1]:
# Parse the current version of YSO

import rdflib
yso = rdflib.Graph()
yso.parse('../vocab/yso-combined-epik.ttl', format='turtle')
print(len(yso))  # show number of triples

589210


In [2]:
# Define helper functions

import functools
import glob
from rdflib.namespace import RDF, OWL, DCTERMS, SKOS

def parse_tsv(fn):
    with open(fn) as tsvf:
        for line in tsvf:
            line = line.strip()
            if '\t' not in line:
                print("warning: cannot parse line, skipping:", line)
                continue
            uri, label = line.strip().split('\t')    
            uri = uri[1:-1]
            yield (uri, label)

@functools.lru_cache(maxsize=30000)
def concept_label(uri, default, lang):
    labels = yso.preferredLabel(uri, lang=lang)
    if labels:
        return str(labels[0][1])
    else:
        return default

@functools.lru_cache(maxsize=30000)
def replace_concept(subj, lang):
    uri, label = subj
    uri = rdflib.URIRef(uri)
    replacements = list(yso.objects(uri, DCTERMS.isReplacedBy))
    if len(replacements) == 0:
        return [(str(uri), concept_label(uri, label, lang))]
    else:
        print("replacing", uri, "with", ' '.join([str(r) for r in replacements]))
        if len(replacements) > 1:
            print("warning: multiple replacements for", uri)
        return [(str(r), concept_label(r, label, lang)) for r in replacements]

@functools.lru_cache(maxsize=30000)
def check_concept(uri, lang):
    uri = rdflib.URIRef(uri)
    if (uri, RDF.type, SKOS.Concept) not in yso:
        print(uri, "not a skos:Concept")
        return False
    if (uri, OWL.deprecated, True) in yso:
        print(uri, "is deprecated")
        return False
    if concept_label(uri, None, lang) is None:
        print(uri, "has no label")
        return False
    return True


            
def write_tsv(fn, subjects):
    with open(fn, 'w') as tsvf:
        for uri, label in subjects:
            print("<{}>\t{}".format(uri, label), file=tsvf)

def sync_fulltext_dir(pattern, lang):
    for idx, fn in enumerate(glob.glob(pattern)):
        print(fn)
        l = list(parse_tsv(fn))
        #print(l)
        subjects = []
        for subj in l:
            subjects += replace_concept(subj, lang)
        for uri, label in subjects:
            check_concept(uri, lang)
        #print(subjects)
        write_tsv(fn, subjects)


In [3]:
sync_fulltext_dir('../fulltext/kirjastonhoitaja/*/*.tsv', 'fi')

../fulltext/kirjastonhoitaja/all/384515.tsv
http://www.yso.fi/onto/yso/p2223 not a skos:Concept
http://www.yso.fi/onto/yso/p6090 not a skos:Concept
http://www.yso.fi/onto/yso/p15421 not a skos:Concept
http://www.yso.fi/onto/yso/p14481 not a skos:Concept
../fulltext/kirjastonhoitaja/all/384557.tsv
http://www.yso.fi/onto/yso/p12687 not a skos:Concept
http://www.yso.fi/onto/yso/p386 not a skos:Concept
http://www.yso.fi/onto/yso/p18715 not a skos:Concept
http://www.yso.fi/onto/yso/p2371 not a skos:Concept
../fulltext/kirjastonhoitaja/all/384583.tsv
http://www.yso.fi/onto/yso/p4742 not a skos:Concept
http://www.yso.fi/onto/yso/p15598 not a skos:Concept
http://www.yso.fi/onto/yso/p5398 not a skos:Concept
http://www.yso.fi/onto/yso/p8970 not a skos:Concept
http://www.yso.fi/onto/yso/p8416 not a skos:Concept
../fulltext/kirjastonhoitaja/all/384590.tsv
http://www.yso.fi/onto/yso/p20004 not a skos:Concept
http://www.yso.fi/onto/yso/p12012 not a skos:Concept
http://www.yso.fi/onto/yso/p11808 not 

http://www.yso.fi/onto/yso/p3910 not a skos:Concept
http://www.yso.fi/onto/yso/p8173 not a skos:Concept
http://www.yso.fi/onto/yso/p3968 not a skos:Concept
http://www.yso.fi/onto/yso/p763 not a skos:Concept
../fulltext/kirjastonhoitaja/all/385457.tsv
http://www.yso.fi/onto/yso/p18452 not a skos:Concept
http://www.yso.fi/onto/yso/p2921 not a skos:Concept
http://www.yso.fi/onto/yso/p2559 not a skos:Concept
http://www.yso.fi/onto/yso/p15408 not a skos:Concept
../fulltext/kirjastonhoitaja/all/385485.tsv
http://www.yso.fi/onto/yso/p9664 not a skos:Concept
http://www.yso.fi/onto/yso/p3001 not a skos:Concept
http://www.yso.fi/onto/yso/p4797 not a skos:Concept
../fulltext/kirjastonhoitaja/all/385493.tsv
http://www.yso.fi/onto/yso/p3490 not a skos:Concept
http://www.yso.fi/onto/yso/p24644 not a skos:Concept
../fulltext/kirjastonhoitaja/all/385498.tsv
http://www.yso.fi/onto/yso/p18693 not a skos:Concept
../fulltext/kirjastonhoitaja/all/385509.tsv
http://www.yso.fi/onto/yso/p2571 not a skos:Conce

http://www.yso.fi/onto/yso/p261 not a skos:Concept
http://www.yso.fi/onto/yso/p2070 not a skos:Concept
http://www.yso.fi/onto/yso/p862 not a skos:Concept
http://www.yso.fi/onto/yso/p856 not a skos:Concept
../fulltext/kirjastonhoitaja/all/386419.tsv
http://www.yso.fi/onto/yso/p3860 not a skos:Concept
http://www.yso.fi/onto/yso/p3670 not a skos:Concept
http://www.yso.fi/onto/yso/p2360 not a skos:Concept
http://www.yso.fi/onto/yso/p11781 not a skos:Concept
http://www.yso.fi/onto/yso/p4747 not a skos:Concept
http://www.yso.fi/onto/yso/p4257 not a skos:Concept
../fulltext/kirjastonhoitaja/all/386420.tsv
http://www.yso.fi/onto/yso/p6505 not a skos:Concept
../fulltext/kirjastonhoitaja/all/386459.tsv
http://www.yso.fi/onto/yso/p7915 not a skos:Concept
http://www.yso.fi/onto/yso/p7727 not a skos:Concept
http://www.yso.fi/onto/yso/p5889 not a skos:Concept
http://www.yso.fi/onto/yso/p2211 not a skos:Concept
../fulltext/kirjastonhoitaja/all/386463.tsv
http://www.yso.fi/onto/yso/p10632 not a skos:C

../fulltext/kirjastonhoitaja/all/388413.tsv
http://www.yso.fi/onto/yso/p18011 not a skos:Concept
http://www.yso.fi/onto/yso/p5087 not a skos:Concept
../fulltext/kirjastonhoitaja/all/388419.tsv
http://www.yso.fi/onto/yso/p17179 not a skos:Concept
http://www.yso.fi/onto/yso/p11116 not a skos:Concept
http://www.yso.fi/onto/yso/p13151 not a skos:Concept
../fulltext/kirjastonhoitaja/all/388430.tsv
http://www.yso.fi/onto/yso/p2088 not a skos:Concept
../fulltext/kirjastonhoitaja/all/388464.tsv
http://www.yso.fi/onto/yso/p1599 not a skos:Concept
http://www.yso.fi/onto/yso/p9426 not a skos:Concept
http://www.yso.fi/onto/yso/p12629 not a skos:Concept
../fulltext/kirjastonhoitaja/all/388482.tsv
http://www.yso.fi/onto/yso/p4150 not a skos:Concept
http://www.yso.fi/onto/yso/p16484 not a skos:Concept
http://www.yso.fi/onto/yso/p17133 not a skos:Concept
http://www.yso.fi/onto/yso/p18200 not a skos:Concept
../fulltext/kirjastonhoitaja/all/388528.tsv
http://www.yso.fi/onto/yso/p2785 not a skos:Concept


../fulltext/kirjastonhoitaja/all/390463.tsv
http://www.yso.fi/onto/yso/p2408 not a skos:Concept
http://www.yso.fi/onto/yso/p2692 not a skos:Concept
../fulltext/kirjastonhoitaja/all/390532.tsv
http://www.yso.fi/onto/yso/p3708 not a skos:Concept
../fulltext/kirjastonhoitaja/all/390570.tsv
../fulltext/kirjastonhoitaja/all/390592.tsv
http://www.yso.fi/onto/yso/p11275 not a skos:Concept
http://www.yso.fi/onto/yso/p924 not a skos:Concept
http://www.yso.fi/onto/yso/p2683 not a skos:Concept
http://www.yso.fi/onto/yso/p16009 not a skos:Concept
../fulltext/kirjastonhoitaja/all/390633.tsv
http://www.yso.fi/onto/yso/p8531 not a skos:Concept
http://www.yso.fi/onto/yso/p9517 not a skos:Concept
../fulltext/kirjastonhoitaja/all/390663.tsv
http://www.yso.fi/onto/yso/p8251 not a skos:Concept
http://www.yso.fi/onto/yso/p4328 not a skos:Concept
http://www.yso.fi/onto/yso/p8772 not a skos:Concept
http://www.yso.fi/onto/yso/p5208 not a skos:Concept
http://www.yso.fi/onto/yso/p4343 not a skos:Concept
http://

...

../fulltext/kirjastonhoitaja/train/428059.tsv
../fulltext/kirjastonhoitaja/train/428060.tsv
../fulltext/kirjastonhoitaja/train/428062.tsv
../fulltext/kirjastonhoitaja/train/428063.tsv
../fulltext/kirjastonhoitaja/train/428156.tsv
../fulltext/kirjastonhoitaja/train/428188.tsv
../fulltext/kirjastonhoitaja/train/428196.tsv
../fulltext/kirjastonhoitaja/train/428241.tsv
../fulltext/kirjastonhoitaja/train/428245.tsv
../fulltext/kirjastonhoitaja/train/428285.tsv
../fulltext/kirjastonhoitaja/train/428355.tsv
../fulltext/kirjastonhoitaja/train/428433.tsv
../fulltext/kirjastonhoitaja/train/428454.tsv
../fulltext/kirjastonhoitaja/train/428460.tsv
../fulltext/kirjastonhoitaja/train/428461.tsv
../fulltext/kirjastonhoitaja/train/428469.tsv
../fulltext/kirjastonhoitaja/train/428487.tsv
../fulltext/kirjastonhoitaja/train/428542.tsv
../fulltext/kirjastonhoitaja/train/428554.tsv
../fulltext/kirjastonhoitaja/train/428564.tsv
../fulltext/kirjastonhoitaja/train/428581.tsv
../fulltext/kirjastonhoitaja/train

../fulltext/kirjastonhoitaja/train/431554.tsv
../fulltext/kirjastonhoitaja/train/431588.tsv
../fulltext/kirjastonhoitaja/train/431592.tsv
../fulltext/kirjastonhoitaja/train/431625.tsv
../fulltext/kirjastonhoitaja/train/431628.tsv
../fulltext/kirjastonhoitaja/train/431632.tsv
../fulltext/kirjastonhoitaja/train/431633.tsv
../fulltext/kirjastonhoitaja/train/431638.tsv
../fulltext/kirjastonhoitaja/train/431643.tsv
../fulltext/kirjastonhoitaja/train/431669.tsv
../fulltext/kirjastonhoitaja/train/431700.tsv
../fulltext/kirjastonhoitaja/train/431704.tsv
../fulltext/kirjastonhoitaja/train/431747.tsv
../fulltext/kirjastonhoitaja/train/431748.tsv
../fulltext/kirjastonhoitaja/train/431751.tsv
../fulltext/kirjastonhoitaja/train/431756.tsv
../fulltext/kirjastonhoitaja/train/431776.tsv
../fulltext/kirjastonhoitaja/train/431821.tsv
../fulltext/kirjastonhoitaja/train/431828.tsv
../fulltext/kirjastonhoitaja/train/431831.tsv
../fulltext/kirjastonhoitaja/train/431857.tsv
../fulltext/kirjastonhoitaja/train

../fulltext/kirjastonhoitaja/train/434320.tsv
../fulltext/kirjastonhoitaja/train/434331.tsv
../fulltext/kirjastonhoitaja/train/434344.tsv
../fulltext/kirjastonhoitaja/train/434356.tsv
../fulltext/kirjastonhoitaja/train/434369.tsv
../fulltext/kirjastonhoitaja/train/434371.tsv
../fulltext/kirjastonhoitaja/train/434378.tsv
../fulltext/kirjastonhoitaja/train/434386.tsv
../fulltext/kirjastonhoitaja/train/434394.tsv
../fulltext/kirjastonhoitaja/train/434409.tsv
../fulltext/kirjastonhoitaja/train/434411.tsv
../fulltext/kirjastonhoitaja/train/434430.tsv
../fulltext/kirjastonhoitaja/train/434431.tsv
../fulltext/kirjastonhoitaja/train/434450.tsv
../fulltext/kirjastonhoitaja/train/434460.tsv
../fulltext/kirjastonhoitaja/train/434467.tsv
../fulltext/kirjastonhoitaja/train/434472.tsv
../fulltext/kirjastonhoitaja/train/434479.tsv
../fulltext/kirjastonhoitaja/train/434481.tsv
../fulltext/kirjastonhoitaja/train/434494.tsv
../fulltext/kirjastonhoitaja/train/434495.tsv
../fulltext/kirjastonhoitaja/train

../fulltext/kirjastonhoitaja/train/436296.tsv
../fulltext/kirjastonhoitaja/train/436333.tsv
../fulltext/kirjastonhoitaja/train/436338.tsv
../fulltext/kirjastonhoitaja/train/436341.tsv
../fulltext/kirjastonhoitaja/train/436347.tsv
../fulltext/kirjastonhoitaja/train/436351.tsv
../fulltext/kirjastonhoitaja/train/436355.tsv
../fulltext/kirjastonhoitaja/train/436357.tsv
../fulltext/kirjastonhoitaja/train/436364.tsv
../fulltext/kirjastonhoitaja/train/436387.tsv
../fulltext/kirjastonhoitaja/train/436394.tsv
../fulltext/kirjastonhoitaja/train/436395.tsv
../fulltext/kirjastonhoitaja/train/436419.tsv
../fulltext/kirjastonhoitaja/train/436432.tsv
../fulltext/kirjastonhoitaja/train/436438.tsv
../fulltext/kirjastonhoitaja/train/436446.tsv
../fulltext/kirjastonhoitaja/train/436450.tsv
../fulltext/kirjastonhoitaja/train/436458.tsv
../fulltext/kirjastonhoitaja/train/436463.tsv
../fulltext/kirjastonhoitaja/train/436471.tsv
../fulltext/kirjastonhoitaja/train/436478.tsv
../fulltext/kirjastonhoitaja/train

../fulltext/kirjastonhoitaja/validate/438669.tsv
../fulltext/kirjastonhoitaja/validate/438677.tsv
../fulltext/kirjastonhoitaja/validate/438703.tsv
../fulltext/kirjastonhoitaja/validate/438722.tsv
../fulltext/kirjastonhoitaja/validate/438735.tsv
../fulltext/kirjastonhoitaja/validate/438747.tsv
../fulltext/kirjastonhoitaja/validate/438763.tsv
../fulltext/kirjastonhoitaja/validate/438765.tsv
../fulltext/kirjastonhoitaja/validate/438840.tsv
../fulltext/kirjastonhoitaja/validate/438848.tsv
../fulltext/kirjastonhoitaja/validate/438873.tsv
../fulltext/kirjastonhoitaja/validate/438922.tsv
../fulltext/kirjastonhoitaja/validate/438962.tsv
../fulltext/kirjastonhoitaja/validate/438964.tsv
../fulltext/kirjastonhoitaja/validate/438967.tsv
../fulltext/kirjastonhoitaja/validate/438974.tsv
../fulltext/kirjastonhoitaja/validate/439014.tsv
../fulltext/kirjastonhoitaja/validate/439051.tsv
../fulltext/kirjastonhoitaja/validate/439109.tsv
../fulltext/kirjastonhoitaja/validate/439117.tsv
../fulltext/kirjasto

In [4]:
sync_fulltext_dir('../fulltext/jyu-theses/fin/*.tsv', 'fi')

../fulltext/jyu-theses/fin/2010-D-22709.tsv
http://www.yso.fi/onto/yso/p6858 not a skos:Concept
http://www.yso.fi/onto/yso/p19247 not a skos:Concept
http://www.yso.fi/onto/yso/p6939 not a skos:Concept
http://www.yso.fi/onto/yso/p3402 not a skos:Concept
http://www.yso.fi/onto/yso/p16315 not a skos:Concept
http://www.yso.fi/onto/yso/p9695 not a skos:Concept
../fulltext/jyu-theses/fin/2010-D-22711.tsv
http://www.yso.fi/onto/yso/p11748 not a skos:Concept
http://www.yso.fi/onto/yso/p9728 not a skos:Concept
../fulltext/jyu-theses/fin/2010-D-22712.tsv
http://www.yso.fi/onto/yso/p10164 not a skos:Concept
../fulltext/jyu-theses/fin/2010-D-22736.tsv
http://www.yso.fi/onto/yso/p10372 not a skos:Concept
http://www.yso.fi/onto/yso/p6985 not a skos:Concept
../fulltext/jyu-theses/fin/2010-D-22737.tsv
../fulltext/jyu-theses/fin/2010-D-23021.tsv
http://www.yso.fi/onto/yso/p12736 not a skos:Concept
http://www.yso.fi/onto/yso/p11694 not a skos:Concept
http://www.yso.fi/onto/yso/p1068 not a skos:Concept
h

http://www.yso.fi/onto/yso/p24266 not a skos:Concept
http://www.yso.fi/onto/yso/p18355 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-23062.tsv
http://www.yso.fi/onto/yso/p13189 not a skos:Concept
http://www.yso.fi/onto/yso/p17897 not a skos:Concept
http://www.yso.fi/onto/yso/p18676 not a skos:Concept
http://www.yso.fi/onto/yso/p15263 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-23065.tsv
http://www.yso.fi/onto/yso/p21142 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-23067.tsv
http://www.yso.fi/onto/yso/p21811 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-23084.tsv
http://www.yso.fi/onto/yso/p13479 not a skos:Concept
http://www.yso.fi/onto/yso/p21 not a skos:Concept
http://www.yso.fi/onto/yso/p11912 not a skos:Concept
http://www.yso.fi/onto/yso/p22619 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-23086.tsv
http://www.yso.fi/onto/yso/p3751 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-23182.tsv
../fulltext/jyu-theses/fin/2010-M-23213.tsv
http://

../fulltext/jyu-theses/fin/2010-M-24546.tsv
../fulltext/jyu-theses/fin/2010-M-24547.tsv
http://www.yso.fi/onto/yso/p13332 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-24555.tsv
http://www.yso.fi/onto/yso/p19158 not a skos:Concept
http://www.yso.fi/onto/yso/p15164 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-24559.tsv
http://www.yso.fi/onto/yso/p3999 not a skos:Concept
http://www.yso.fi/onto/yso/p22016 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-24571.tsv
http://www.yso.fi/onto/yso/p16286 not a skos:Concept
http://www.yso.fi/onto/yso/p9738 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-24572.tsv
http://www.yso.fi/onto/yso/p23578 not a skos:Concept
http://www.yso.fi/onto/yso/p3062 not a skos:Concept
http://www.yso.fi/onto/yso/p9953 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-24573.tsv
http://www.yso.fi/onto/yso/p11440 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-24574.tsv
http://www.yso.fi/onto/yso/p7384 not a skos:Concept
http://www.yso.fi

http://www.yso.fi/onto/yso/p17342 not a skos:Concept
http://www.yso.fi/onto/yso/p7089 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-25285.tsv
../fulltext/jyu-theses/fin/2010-M-25286.tsv
http://www.yso.fi/onto/yso/p16518 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-25307.tsv
http://www.yso.fi/onto/yso/p20098 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-25316.tsv
http://www.yso.fi/onto/yso/p8478 not a skos:Concept
http://www.yso.fi/onto/yso/p17473 not a skos:Concept
http://www.yso.fi/onto/yso/p18770 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-25348.tsv
http://www.yso.fi/onto/yso/p19679 not a skos:Concept
http://www.yso.fi/onto/yso/p20924 not a skos:Concept
http://www.yso.fi/onto/yso/p812 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-25359.tsv
http://www.yso.fi/onto/yso/p14536 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-25389.tsv
http://www.yso.fi/onto/yso/p4635 not a skos:Concept
http://www.yso.fi/onto/yso/p5677 not a skos:Concept
http://ww

../fulltext/jyu-theses/fin/2010-M-25697.tsv
http://www.yso.fi/onto/yso/p24064 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-25701.tsv
../fulltext/jyu-theses/fin/2010-M-25702.tsv
../fulltext/jyu-theses/fin/2010-M-25703.tsv
../fulltext/jyu-theses/fin/2010-M-25704.tsv
http://www.yso.fi/onto/yso/p19780 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-25705.tsv
http://www.yso.fi/onto/yso/p6840 not a skos:Concept
http://www.yso.fi/onto/yso/p6841 not a skos:Concept
http://www.yso.fi/onto/yso/p23734 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-25709.tsv
http://www.yso.fi/onto/yso/p9569 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-25713.tsv
http://www.yso.fi/onto/yso/p22220 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-25718.tsv
http://www.yso.fi/onto/yso/p7656 not a skos:Concept
http://www.yso.fi/onto/yso/p108 not a skos:Concept
../fulltext/jyu-theses/fin/2010-M-25720.tsv
http://www.yso.fi/onto/yso/p7678 not a skos:Concept
http://www.yso.fi/onto/yso/p23208 not

...

../fulltext/jyu-theses/fin/2017-M-54005.tsv
../fulltext/jyu-theses/fin/2017-M-54014.tsv
../fulltext/jyu-theses/fin/2017-M-54025.tsv
../fulltext/jyu-theses/fin/2017-M-54026.tsv
../fulltext/jyu-theses/fin/2017-M-54029.tsv
../fulltext/jyu-theses/fin/2017-M-54031.tsv
http://www.yso.fi/onto/yso/p926 not a skos:Concept
http://www.yso.fi/onto/yso/p4727 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-54032.tsv
../fulltext/jyu-theses/fin/2017-M-54033.tsv
../fulltext/jyu-theses/fin/2017-M-54035.tsv
http://www.yso.fi/onto/yso/p3851 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-54042.tsv
../fulltext/jyu-theses/fin/2017-M-54048.tsv
../fulltext/jyu-theses/fin/2017-M-54057.tsv
../fulltext/jyu-theses/fin/2017-M-54061.tsv
http://www.yso.fi/onto/yso/p10993 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-54064.tsv
../fulltext/jyu-theses/fin/2017-M-54067.tsv
../fulltext/jyu-theses/fin/2017-M-54074.tsv
http://www.yso.fi/onto/yso/p13051 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-5

../fulltext/jyu-theses/fin/2017-M-54508.tsv
../fulltext/jyu-theses/fin/2017-M-54517.tsv
http://www.yso.fi/onto/yso/p5122 not a skos:Concept
http://www.yso.fi/onto/yso/p29709 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-54520.tsv
http://www.yso.fi/onto/yso/p171 not a skos:Concept
http://www.yso.fi/onto/yso/p27723 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-54521.tsv
../fulltext/jyu-theses/fin/2017-M-54522.tsv
http://www.yso.fi/onto/yso/p5915 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-54523.tsv
../fulltext/jyu-theses/fin/2017-M-54526.tsv
../fulltext/jyu-theses/fin/2017-M-54529.tsv
../fulltext/jyu-theses/fin/2017-M-54531.tsv
../fulltext/jyu-theses/fin/2017-M-54532.tsv
http://www.yso.fi/onto/yso/p3854 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-54534.tsv
../fulltext/jyu-theses/fin/2017-M-54535.tsv
../fulltext/jyu-theses/fin/2017-M-54540.tsv
http://www.yso.fi/onto/yso/p8003 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-54542.tsv
../fulltext/jyu-the

http://www.yso.fi/onto/yso/p28380 not a skos:Concept
http://www.yso.fi/onto/yso/p16014 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-55316.tsv
http://www.yso.fi/onto/yso/p8709 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-55321.tsv
../fulltext/jyu-theses/fin/2017-M-55323.tsv
../fulltext/jyu-theses/fin/2017-M-55324.tsv
http://www.yso.fi/onto/yso/p3214 not a skos:Concept
http://www.yso.fi/onto/yso/p5992 not a skos:Concept
http://www.yso.fi/onto/yso/p1813 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-55325.tsv
http://www.yso.fi/onto/yso/p16197 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-55332.tsv
http://www.yso.fi/onto/yso/p10172 not a skos:Concept
http://www.yso.fi/onto/yso/p10755 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-55369.tsv
../fulltext/jyu-theses/fin/2017-M-55374.tsv
../fulltext/jyu-theses/fin/2017-M-55375.tsv
../fulltext/jyu-theses/fin/2017-M-55376.tsv
../fulltext/jyu-theses/fin/2017-M-55384.tsv
../fulltext/jyu-theses/fin/2017-M-55389.tsv

../fulltext/jyu-theses/fin/2017-M-55886.tsv
http://www.yso.fi/onto/yso/p15494 not a skos:Concept
http://www.yso.fi/onto/yso/p6772 not a skos:Concept
http://www.yso.fi/onto/yso/p16085 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-55887.tsv
http://www.yso.fi/onto/yso/p27738 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-55888.tsv
http://www.yso.fi/onto/yso/p10393 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-55906.tsv
../fulltext/jyu-theses/fin/2017-M-55911.tsv
../fulltext/jyu-theses/fin/2017-M-55912.tsv
http://www.yso.fi/onto/yso/p19379 not a skos:Concept
http://www.yso.fi/onto/yso/p29647 not a skos:Concept
http://www.yso.fi/onto/yso/p25268 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-55914.tsv
http://www.yso.fi/onto/yso/p10774 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-55918.tsv
../fulltext/jyu-theses/fin/2017-M-55921.tsv
../fulltext/jyu-theses/fin/2017-M-55923.tsv
../fulltext/jyu-theses/fin/2017-M-55930.tsv
http://www.yso.fi/onto/yso/p23709 not a 

../fulltext/jyu-theses/fin/2017-M-56769.tsv
../fulltext/jyu-theses/fin/2017-M-56775.tsv
../fulltext/jyu-theses/fin/2017-M-56780.tsv
../fulltext/jyu-theses/fin/2017-M-56793.tsv
http://www.yso.fi/onto/yso/p27697 not a skos:Concept
http://www.yso.fi/onto/yso/p2131 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-56795.tsv
../fulltext/jyu-theses/fin/2017-M-56824.tsv
../fulltext/jyu-theses/fin/2017-M-56838.tsv
http://www.yso.fi/onto/yso/p29580 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-56863.tsv
../fulltext/jyu-theses/fin/2017-M-56870.tsv
http://www.yso.fi/onto/yso/p6588 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-56892.tsv
../fulltext/jyu-theses/fin/2017-M-56897.tsv
http://www.yso.fi/onto/yso/p6206 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-56907.tsv
../fulltext/jyu-theses/fin/2017-M-56908.tsv
../fulltext/jyu-theses/fin/2017-M-56921.tsv
http://www.yso.fi/onto/yso/p19091 not a skos:Concept
../fulltext/jyu-theses/fin/2017-M-56926.tsv
../fulltext/jyu-theses/fi

In [5]:
sync_fulltext_dir('../fulltext/jyu-theses/swe/*.tsv', 'sv')

../fulltext/jyu-theses/swe/2010-M-23239.tsv
http://www.yso.fi/onto/yso/p5303 not a skos:Concept
http://www.yso.fi/onto/yso/p5301 not a skos:Concept
http://www.yso.fi/onto/yso/p10746 not a skos:Concept
http://www.yso.fi/onto/yso/p21292 not a skos:Concept
../fulltext/jyu-theses/swe/2010-M-24757.tsv
http://www.yso.fi/onto/yso/p15724 not a skos:Concept
http://www.yso.fi/onto/yso/p19214 not a skos:Concept
http://www.yso.fi/onto/yso/p6726 not a skos:Concept
http://www.yso.fi/onto/yso/p13463 not a skos:Concept
../fulltext/jyu-theses/swe/2010-M-24861.tsv
http://www.yso.fi/onto/yso/p7416 not a skos:Concept
http://www.yso.fi/onto/yso/p556 not a skos:Concept
http://www.yso.fi/onto/yso/p2945 not a skos:Concept
http://www.yso.fi/onto/yso/p2630 not a skos:Concept
../fulltext/jyu-theses/swe/2010-M-24881.tsv
http://www.yso.fi/onto/yso/p2736 not a skos:Concept
http://www.yso.fi/onto/yso/p5797 not a skos:Concept
http://www.yso.fi/onto/yso/p12998 not a skos:Concept
../fulltext/jyu-theses/swe/2010-M-25283

http://www.yso.fi/onto/yso/p159 not a skos:Concept
http://www.yso.fi/onto/yso/p3929 not a skos:Concept
../fulltext/jyu-theses/swe/2013-M-42451.tsv
http://www.yso.fi/onto/yso/p2095 not a skos:Concept
http://www.yso.fi/onto/yso/p1798 not a skos:Concept
../fulltext/jyu-theses/swe/2013-M-42462.tsv
http://www.yso.fi/onto/yso/p4070 not a skos:Concept
http://www.yso.fi/onto/yso/p6164 not a skos:Concept
../fulltext/jyu-theses/swe/2013-M-42500.tsv
../fulltext/jyu-theses/swe/2013-M-42594.tsv
http://www.yso.fi/onto/yso/p17817 not a skos:Concept
http://www.yso.fi/onto/yso/p1407 not a skos:Concept
http://www.yso.fi/onto/yso/p417 not a skos:Concept
../fulltext/jyu-theses/swe/2013-M-42608.tsv
http://www.yso.fi/onto/yso/p25701 not a skos:Concept
http://www.yso.fi/onto/yso/p6545 not a skos:Concept
http://www.yso.fi/onto/yso/p8744 not a skos:Concept
http://www.yso.fi/onto/yso/p11392 not a skos:Concept
http://www.yso.fi/onto/yso/p9239 not a skos:Concept
../fulltext/jyu-theses/swe/2013-M-42616.tsv
http://

../fulltext/jyu-theses/swe/2016-M-51330.tsv
http://www.yso.fi/onto/yso/p11735 not a skos:Concept
../fulltext/jyu-theses/swe/2016-M-52290.tsv
http://www.yso.fi/onto/yso/p26073 not a skos:Concept
../fulltext/jyu-theses/swe/2016-M-52349.tsv
../fulltext/jyu-theses/swe/2016-M-52635.tsv
http://www.yso.fi/onto/yso/p15764 not a skos:Concept
http://www.yso.fi/onto/yso/p2921 not a skos:Concept
../fulltext/jyu-theses/swe/2016-M-52646.tsv
http://www.yso.fi/onto/yso/p2494 not a skos:Concept
../fulltext/jyu-theses/swe/2017-D-54078.tsv
http://www.yso.fi/onto/yso/p36 not a skos:Concept
http://www.yso.fi/onto/yso/p1353 not a skos:Concept
http://www.yso.fi/onto/yso/p7831 not a skos:Concept
../fulltext/jyu-theses/swe/2017-M-53640.tsv
http://www.yso.fi/onto/yso/p8692 not a skos:Concept
../fulltext/jyu-theses/swe/2017-M-53710.tsv
http://www.yso.fi/onto/yso/p559 not a skos:Concept
../fulltext/jyu-theses/swe/2017-M-54174.tsv
http://www.yso.fi/onto/yso/p7111 not a skos:Concept
http://www.yso.fi/onto/yso/p1202

In [6]:
sync_fulltext_dir('../fulltext/jyu-theses/eng/*.tsv', 'en')

../fulltext/jyu-theses/eng/2010-D-22710.tsv
http://www.yso.fi/onto/yso/p12732 not a skos:Concept
http://www.yso.fi/onto/yso/p12733 not a skos:Concept
http://www.yso.fi/onto/yso/p2041 not a skos:Concept
http://www.yso.fi/onto/yso/p11617 not a skos:Concept
http://www.yso.fi/onto/yso/p3734 not a skos:Concept
http://www.yso.fi/onto/yso/p19861 not a skos:Concept
http://www.yso.fi/onto/yso/p4513 not a skos:Concept
../fulltext/jyu-theses/eng/2010-D-22733.tsv
http://www.yso.fi/onto/yso/p9457 not a skos:Concept
http://www.yso.fi/onto/yso/p8246 not a skos:Concept
http://www.yso.fi/onto/yso/p5029 not a skos:Concept
../fulltext/jyu-theses/eng/2010-D-22922.tsv
http://www.yso.fi/onto/yso/p9327 not a skos:Concept
http://www.yso.fi/onto/yso/p23081 not a skos:Concept
http://www.yso.fi/onto/yso/p15854 not a skos:Concept
http://www.yso.fi/onto/yso/p1181 not a skos:Concept
http://www.yso.fi/onto/yso/p854 not a skos:Concept
http://www.yso.fi/onto/yso/p3615 not a skos:Concept
http://www.yso.fi/onto/yso/p113

http://www.yso.fi/onto/yso/p2784 not a skos:Concept
http://www.yso.fi/onto/yso/p14975 not a skos:Concept
http://www.yso.fi/onto/yso/p20176 not a skos:Concept
http://www.yso.fi/onto/yso/p3920 not a skos:Concept
http://www.yso.fi/onto/yso/p128 not a skos:Concept
http://www.yso.fi/onto/yso/p26412 not a skos:Concept
../fulltext/jyu-theses/eng/2010-D-24893.tsv
http://www.yso.fi/onto/yso/p2410 not a skos:Concept
http://www.yso.fi/onto/yso/p136 not a skos:Concept
../fulltext/jyu-theses/eng/2010-D-24903.tsv
http://www.yso.fi/onto/yso/p9514 not a skos:Concept
http://www.yso.fi/onto/yso/p7758 not a skos:Concept
http://www.yso.fi/onto/yso/p12661 not a skos:Concept
http://www.yso.fi/onto/yso/p11788 not a skos:Concept
http://www.yso.fi/onto/yso/p24041 not a skos:Concept
http://www.yso.fi/onto/yso/p2317 not a skos:Concept
../fulltext/jyu-theses/eng/2010-D-24904.tsv
http://www.yso.fi/onto/yso/p2407 not a skos:Concept
http://www.yso.fi/onto/yso/p4787 not a skos:Concept
http://www.yso.fi/onto/yso/p1347

../fulltext/jyu-theses/eng/2010-D-25738.tsv
http://www.yso.fi/onto/yso/p7621 not a skos:Concept
http://www.yso.fi/onto/yso/p10083 not a skos:Concept
http://www.yso.fi/onto/yso/p7569 not a skos:Concept
http://www.yso.fi/onto/yso/p20083 not a skos:Concept
../fulltext/jyu-theses/eng/2010-D-25741.tsv
http://www.yso.fi/onto/yso/p19421 not a skos:Concept
http://www.yso.fi/onto/yso/p6548 not a skos:Concept
http://www.yso.fi/onto/yso/p15898 not a skos:Concept
http://www.yso.fi/onto/yso/p15899 not a skos:Concept
http://www.yso.fi/onto/yso/p1127 not a skos:Concept
http://www.yso.fi/onto/yso/p21989 not a skos:Concept
http://www.yso.fi/onto/yso/p12928 not a skos:Concept
http://www.yso.fi/onto/yso/p3542 not a skos:Concept
http://www.yso.fi/onto/yso/p1949 not a skos:Concept
http://www.yso.fi/onto/yso/p1946 not a skos:Concept
http://www.yso.fi/onto/yso/p9860 not a skos:Concept
http://www.yso.fi/onto/yso/p84 not a skos:Concept
http://www.yso.fi/onto/yso/p1912 not a skos:Concept
http://www.yso.fi/onto/

http://www.yso.fi/onto/yso/p11094 not a skos:Concept
http://www.yso.fi/onto/yso/p23026 not a skos:Concept
../fulltext/jyu-theses/eng/2010-M-25491.tsv
http://www.yso.fi/onto/yso/p9484 not a skos:Concept
../fulltext/jyu-theses/eng/2010-M-25498.tsv
http://www.yso.fi/onto/yso/p1759 not a skos:Concept
http://www.yso.fi/onto/yso/p1832 not a skos:Concept
http://www.yso.fi/onto/yso/p21319 not a skos:Concept
../fulltext/jyu-theses/eng/2010-M-25502.tsv
http://www.yso.fi/onto/yso/p3867 not a skos:Concept
http://www.yso.fi/onto/yso/p13777 not a skos:Concept
http://www.yso.fi/onto/yso/p2209 not a skos:Concept
../fulltext/jyu-theses/eng/2010-M-25505.tsv
http://www.yso.fi/onto/yso/p157 not a skos:Concept
http://www.yso.fi/onto/yso/p7830 not a skos:Concept
http://www.yso.fi/onto/yso/p14612 not a skos:Concept
../fulltext/jyu-theses/eng/2010-M-25513.tsv
http://www.yso.fi/onto/yso/p8856 not a skos:Concept
http://www.yso.fi/onto/yso/p16486 not a skos:Concept
../fulltext/jyu-theses/eng/2010-M-25519.tsv
htt

../fulltext/jyu-theses/eng/2011-D-27202.tsv
http://www.yso.fi/onto/yso/p12088 not a skos:Concept
http://www.yso.fi/onto/yso/p10177 not a skos:Concept
http://www.yso.fi/onto/yso/p12085 not a skos:Concept
http://www.yso.fi/onto/yso/p12637 not a skos:Concept
http://www.yso.fi/onto/yso/p7756 not a skos:Concept
http://www.yso.fi/onto/yso/p10441 not a skos:Concept
../fulltext/jyu-theses/eng/2011-D-27256.tsv
http://www.yso.fi/onto/yso/p6837 not a skos:Concept
http://www.yso.fi/onto/yso/p9174 not a skos:Concept
http://www.yso.fi/onto/yso/p4393 not a skos:Concept
http://www.yso.fi/onto/yso/p1855 not a skos:Concept
http://www.yso.fi/onto/yso/p23334 not a skos:Concept
http://www.yso.fi/onto/yso/p13809 not a skos:Concept
../fulltext/jyu-theses/eng/2011-D-36548.tsv
http://www.yso.fi/onto/yso/p13599 not a skos:Concept
http://www.yso.fi/onto/yso/p879 not a skos:Concept
http://www.yso.fi/onto/yso/p3169 not a skos:Concept
http://www.yso.fi/onto/yso/p9857 not a skos:Concept
http://www.yso.fi/onto/yso/p4

...

../fulltext/jyu-theses/eng/2016-D-52196.tsv
http://www.yso.fi/onto/yso/p17631 not a skos:Concept
http://www.yso.fi/onto/yso/p29404 not a skos:Concept
http://www.yso.fi/onto/yso/p15360 not a skos:Concept
http://www.yso.fi/onto/yso/p4641 not a skos:Concept
../fulltext/jyu-theses/eng/2016-D-52197.tsv
http://www.yso.fi/onto/yso/p7512 not a skos:Concept
http://www.yso.fi/onto/yso/p24050 not a skos:Concept
../fulltext/jyu-theses/eng/2016-D-52201.tsv
http://www.yso.fi/onto/yso/p17680 not a skos:Concept
http://www.yso.fi/onto/yso/p25129 not a skos:Concept
http://www.yso.fi/onto/yso/p16289 not a skos:Concept
http://www.yso.fi/onto/yso/p14978 not a skos:Concept
../fulltext/jyu-theses/eng/2016-D-52209.tsv
../fulltext/jyu-theses/eng/2016-D-52219.tsv
http://www.yso.fi/onto/yso/p15600 not a skos:Concept
http://www.yso.fi/onto/yso/p9309 not a skos:Concept
http://www.yso.fi/onto/yso/p5860 not a skos:Concept
../fulltext/jyu-theses/eng/2016-D-52220.tsv
http://www.yso.fi/onto/yso/p6369 not a skos:Concept

../fulltext/jyu-theses/eng/2016-M-50492.tsv
../fulltext/jyu-theses/eng/2016-M-50498.tsv
http://www.yso.fi/onto/yso/p17856 not a skos:Concept
../fulltext/jyu-theses/eng/2016-M-50527.tsv
http://www.yso.fi/onto/yso/p18733 not a skos:Concept
http://www.yso.fi/onto/yso/p13084 not a skos:Concept
http://www.yso.fi/onto/yso/p7482 not a skos:Concept
../fulltext/jyu-theses/eng/2016-M-50552.tsv
http://www.yso.fi/onto/yso/p9681 not a skos:Concept
../fulltext/jyu-theses/eng/2016-M-50573.tsv
http://www.yso.fi/onto/yso/p27422 not a skos:Concept
../fulltext/jyu-theses/eng/2016-M-50574.tsv
../fulltext/jyu-theses/eng/2016-M-50581.tsv
../fulltext/jyu-theses/eng/2016-M-50586.tsv
../fulltext/jyu-theses/eng/2016-M-50588.tsv
../fulltext/jyu-theses/eng/2016-M-50590.tsv
http://www.yso.fi/onto/yso/p20287 not a skos:Concept
../fulltext/jyu-theses/eng/2016-M-50620.tsv
../fulltext/jyu-theses/eng/2016-M-50629.tsv
../fulltext/jyu-theses/eng/2016-M-50630.tsv
../fulltext/jyu-theses/eng/2016-M-50634.tsv
../fulltext/jyu

../fulltext/jyu-theses/eng/2016-M-52671.tsv
../fulltext/jyu-theses/eng/2016-M-52729.tsv
http://www.yso.fi/onto/yso/p25974 not a skos:Concept
../fulltext/jyu-theses/eng/2016-M-52756.tsv
../fulltext/jyu-theses/eng/2016-M-52870.tsv
http://www.yso.fi/onto/yso/p15133 not a skos:Concept
../fulltext/jyu-theses/eng/2016-M-52889.tsv
../fulltext/jyu-theses/eng/2016-M-52890.tsv
../fulltext/jyu-theses/eng/2016-M-52945.tsv
http://www.yso.fi/onto/yso/p6356 not a skos:Concept
../fulltext/jyu-theses/eng/2016-M-53002.tsv
http://www.yso.fi/onto/yso/p21007 not a skos:Concept
../fulltext/jyu-theses/eng/2016-M-53038.tsv
http://www.yso.fi/onto/yso/p24563 not a skos:Concept
http://www.yso.fi/onto/yso/p4523 not a skos:Concept
../fulltext/jyu-theses/eng/2016-M-53250.tsv
../fulltext/jyu-theses/eng/2016-M-53832.tsv
../fulltext/jyu-theses/eng/2016-M-54746.tsv
../fulltext/jyu-theses/eng/2016-M-55756.tsv
http://www.yso.fi/onto/yso/p7218 not a skos:Concept
../fulltext/jyu-theses/eng/2017-D-52518.tsv
http://www.yso.f

http://www.yso.fi/onto/yso/p4164 not a skos:Concept
../fulltext/jyu-theses/eng/2017-D-56043.tsv
http://www.yso.fi/onto/yso/p19184 not a skos:Concept
../fulltext/jyu-theses/eng/2017-D-56050.tsv
http://www.yso.fi/onto/yso/p25973 not a skos:Concept
../fulltext/jyu-theses/eng/2017-D-56107.tsv
http://www.yso.fi/onto/yso/p5527 not a skos:Concept
http://www.yso.fi/onto/yso/p26480 not a skos:Concept
../fulltext/jyu-theses/eng/2017-D-56142.tsv
http://www.yso.fi/onto/yso/p12428 not a skos:Concept
../fulltext/jyu-theses/eng/2017-D-56189.tsv
http://www.yso.fi/onto/yso/p2720 not a skos:Concept
../fulltext/jyu-theses/eng/2017-D-56206.tsv
http://www.yso.fi/onto/yso/p20553 not a skos:Concept
http://www.yso.fi/onto/yso/p29101 not a skos:Concept
../fulltext/jyu-theses/eng/2017-D-56207.tsv
http://www.yso.fi/onto/yso/p27907 not a skos:Concept
http://www.yso.fi/onto/yso/p29065 not a skos:Concept
http://www.yso.fi/onto/yso/p27532 not a skos:Concept
../fulltext/jyu-theses/eng/2017-D-56222.tsv
http://www.yso.

http://www.yso.fi/onto/yso/p15019 not a skos:Concept
../fulltext/jyu-theses/eng/2017-M-54666.tsv
http://www.yso.fi/onto/yso/p7957 not a skos:Concept
../fulltext/jyu-theses/eng/2017-M-54674.tsv
http://www.yso.fi/onto/yso/p27575 not a skos:Concept
../fulltext/jyu-theses/eng/2017-M-54692.tsv
http://www.yso.fi/onto/yso/p24555 not a skos:Concept
../fulltext/jyu-theses/eng/2017-M-54694.tsv
http://www.yso.fi/onto/yso/p2586 not a skos:Concept
http://www.yso.fi/onto/yso/p27143 not a skos:Concept
http://www.yso.fi/onto/yso/p1769 not a skos:Concept
../fulltext/jyu-theses/eng/2017-M-54769.tsv
../fulltext/jyu-theses/eng/2017-M-54777.tsv
http://www.yso.fi/onto/yso/p29254 not a skos:Concept
../fulltext/jyu-theses/eng/2017-M-54778.tsv
http://www.yso.fi/onto/yso/p19043 not a skos:Concept
http://www.yso.fi/onto/yso/p23013 not a skos:Concept
../fulltext/jyu-theses/eng/2017-M-54858.tsv
../fulltext/jyu-theses/eng/2017-M-54867.tsv
http://www.yso.fi/onto/yso/p15342 not a skos:Concept
../fulltext/jyu-theses/e

In [7]:
sync_fulltext_dir('../fulltext/arto-fi/all/*.tsv', 'fi')

../fulltext/arto-fi/all/1000213.tsv
../fulltext/arto-fi/all/1000231.tsv
../fulltext/arto-fi/all/1000241.tsv
http://www.yso.fi/onto/yso/p6059 not a skos:Concept
../fulltext/arto-fi/all/1000242.tsv
../fulltext/arto-fi/all/1000249.tsv
../fulltext/arto-fi/all/1000266.tsv
http://www.yso.fi/onto/yso/p11463 not a skos:Concept
../fulltext/arto-fi/all/1000291.tsv
http://www.yso.fi/onto/yso/p17448 not a skos:Concept
http://www.yso.fi/onto/yso/p632 not a skos:Concept
http://www.yso.fi/onto/yso/p7988 not a skos:Concept
../fulltext/arto-fi/all/1001336.tsv
http://www.yso.fi/onto/yso/p8448 not a skos:Concept
../fulltext/arto-fi/all/1001448.tsv
../fulltext/arto-fi/all/1001449.tsv
http://www.yso.fi/onto/yso/p741 not a skos:Concept
../fulltext/arto-fi/all/1001637.tsv
../fulltext/arto-fi/all/1001770.tsv
http://www.yso.fi/onto/yso/p3989 not a skos:Concept
../fulltext/arto-fi/all/1001782.tsv
http://www.yso.fi/onto/yso/p29105 not a skos:Concept
../fulltext/arto-fi/all/1002714.tsv
http://www.yso.fi/onto/yso/

../fulltext/arto-fi/all/1022350.tsv
../fulltext/arto-fi/all/1022357.tsv
http://www.yso.fi/onto/yso/p12756 not a skos:Concept
../fulltext/arto-fi/all/1022369.tsv
../fulltext/arto-fi/all/1022404.tsv
../fulltext/arto-fi/all/1022405.tsv
../fulltext/arto-fi/all/1022410.tsv
../fulltext/arto-fi/all/1022417.tsv
../fulltext/arto-fi/all/1022427.tsv
../fulltext/arto-fi/all/1022429.tsv
../fulltext/arto-fi/all/1022437.tsv
http://www.yso.fi/onto/yso/p5622 not a skos:Concept
../fulltext/arto-fi/all/1022448.tsv
http://www.yso.fi/onto/yso/p12062 not a skos:Concept
../fulltext/arto-fi/all/1022454.tsv
../fulltext/arto-fi/all/1022457.tsv
../fulltext/arto-fi/all/1022468.tsv
../fulltext/arto-fi/all/1022469.tsv
../fulltext/arto-fi/all/1022508.tsv
../fulltext/arto-fi/all/1022509.tsv
../fulltext/arto-fi/all/1022527.tsv
../fulltext/arto-fi/all/1022533.tsv
../fulltext/arto-fi/all/1022553.tsv
http://www.yso.fi/onto/yso/p6527 not a skos:Concept
../fulltext/arto-fi/all/1022558.tsv
../fulltext/arto-fi/all/1022601.ts

../fulltext/arto-fi/all/1029068.tsv
../fulltext/arto-fi/all/1029073.tsv
http://www.yso.fi/onto/yso/p3913 not a skos:Concept
http://www.yso.fi/onto/yso/p6382 not a skos:Concept
../fulltext/arto-fi/all/1029075.tsv
../fulltext/arto-fi/all/1029152.tsv
../fulltext/arto-fi/all/1029172.tsv
http://www.yso.fi/onto/yso/p20420 not a skos:Concept
http://www.yso.fi/onto/yso/p7393 not a skos:Concept
http://www.yso.fi/onto/yso/p8520 not a skos:Concept
../fulltext/arto-fi/all/1029178.tsv
http://www.yso.fi/onto/yso/p11191 not a skos:Concept
http://www.yso.fi/onto/yso/p12969 not a skos:Concept
../fulltext/arto-fi/all/1029196.tsv
http://www.yso.fi/onto/yso/p17775 not a skos:Concept
http://www.yso.fi/onto/yso/p12349 not a skos:Concept
../fulltext/arto-fi/all/1029203.tsv
http://www.yso.fi/onto/yso/p12831 not a skos:Concept
../fulltext/arto-fi/all/1029221.tsv
../fulltext/arto-fi/all/1029322.tsv
http://www.yso.fi/onto/yso/p13965 not a skos:Concept
http://www.yso.fi/onto/yso/p13966 not a skos:Concept
../fullt

http://www.yso.fi/onto/yso/p1371 not a skos:Concept
../fulltext/arto-fi/all/1039381.tsv
../fulltext/arto-fi/all/1039388.tsv
http://www.yso.fi/onto/yso/p1158 not a skos:Concept
http://www.yso.fi/onto/yso/p1156 not a skos:Concept
../fulltext/arto-fi/all/1039390.tsv
../fulltext/arto-fi/all/1039440.tsv
../fulltext/arto-fi/all/1039455.tsv
../fulltext/arto-fi/all/1039532.tsv
../fulltext/arto-fi/all/1039589.tsv
../fulltext/arto-fi/all/1039593.tsv
../fulltext/arto-fi/all/1039596.tsv
../fulltext/arto-fi/all/1039617.tsv
../fulltext/arto-fi/all/1039626.tsv
../fulltext/arto-fi/all/1040049.tsv
http://www.yso.fi/onto/yso/p1176 not a skos:Concept
http://www.yso.fi/onto/yso/p1177 not a skos:Concept
http://www.yso.fi/onto/yso/p3987 not a skos:Concept
../fulltext/arto-fi/all/1040064.tsv
http://www.yso.fi/onto/yso/p10189 not a skos:Concept
../fulltext/arto-fi/all/1040083.tsv
../fulltext/arto-fi/all/1040802.tsv
../fulltext/arto-fi/all/1040841.tsv
../fulltext/arto-fi/all/1040899.tsv
../fulltext/arto-fi/all

../fulltext/arto-fi/all/1076544.tsv
../fulltext/arto-fi/all/1076777.tsv
../fulltext/arto-fi/all/1077321.tsv
../fulltext/arto-fi/all/1077327.tsv
../fulltext/arto-fi/all/1077346.tsv
../fulltext/arto-fi/all/1080447.tsv
../fulltext/arto-fi/all/1083732.tsv
../fulltext/arto-fi/all/1083796.tsv
../fulltext/arto-fi/all/1083797.tsv
http://www.yso.fi/onto/yso/p15059 not a skos:Concept
http://www.yso.fi/onto/yso/p9364 not a skos:Concept
http://www.yso.fi/onto/yso/p16897 not a skos:Concept
../fulltext/arto-fi/all/1083798.tsv
../fulltext/arto-fi/all/1083914.tsv
http://www.yso.fi/onto/yso/p2500 not a skos:Concept
../fulltext/arto-fi/all/1083920.tsv
http://www.yso.fi/onto/yso/p15703 not a skos:Concept
../fulltext/arto-fi/all/1083981.tsv
http://www.yso.fi/onto/yso/p16283 not a skos:Concept
../fulltext/arto-fi/all/1083990.tsv
../fulltext/arto-fi/all/1084115.tsv
../fulltext/arto-fi/all/1084127.tsv
../fulltext/arto-fi/all/1084134.tsv
../fulltext/arto-fi/all/1084139.tsv
../fulltext/arto-fi/all/1084206.tsv


...

http://www.yso.fi/onto/yso/p13807 not a skos:Concept
../fulltext/arto-fi/all/813403.tsv
../fulltext/arto-fi/all/817537.tsv
../fulltext/arto-fi/all/817575.tsv
../fulltext/arto-fi/all/817954.tsv
../fulltext/arto-fi/all/817966.tsv
../fulltext/arto-fi/all/817981.tsv
http://www.yso.fi/onto/yso/p4660 not a skos:Concept
../fulltext/arto-fi/all/817989.tsv
../fulltext/arto-fi/all/817992.tsv
../fulltext/arto-fi/all/818008.tsv
../fulltext/arto-fi/all/818009.tsv
../fulltext/arto-fi/all/818010.tsv
../fulltext/arto-fi/all/818015.tsv
../fulltext/arto-fi/all/818049.tsv
../fulltext/arto-fi/all/818062.tsv
../fulltext/arto-fi/all/818066.tsv
../fulltext/arto-fi/all/818089.tsv
../fulltext/arto-fi/all/818098.tsv
../fulltext/arto-fi/all/818137.tsv
../fulltext/arto-fi/all/818186.tsv
../fulltext/arto-fi/all/818226.tsv
../fulltext/arto-fi/all/818255.tsv
../fulltext/arto-fi/all/818262.tsv
../fulltext/arto-fi/all/818274.tsv
http://www.yso.fi/onto/yso/p5682 not a skos:Concept
../fulltext/arto-fi/all/818281.tsv
../

../fulltext/arto-fi/all/859946.tsv
../fulltext/arto-fi/all/859955.tsv
../fulltext/arto-fi/all/859973.tsv
../fulltext/arto-fi/all/859986.tsv
../fulltext/arto-fi/all/859988.tsv
../fulltext/arto-fi/all/859991.tsv
../fulltext/arto-fi/all/860369.tsv
http://www.yso.fi/onto/yso/p24347 not a skos:Concept
../fulltext/arto-fi/all/860515.tsv
http://www.yso.fi/onto/yso/p25570 not a skos:Concept
http://www.yso.fi/onto/yso/p7525 not a skos:Concept
../fulltext/arto-fi/all/860810.tsv
../fulltext/arto-fi/all/860826.tsv
../fulltext/arto-fi/all/860829.tsv
../fulltext/arto-fi/all/860843.tsv
../fulltext/arto-fi/all/860847.tsv
http://www.yso.fi/onto/yso/p4926 not a skos:Concept
../fulltext/arto-fi/all/860979.tsv
../fulltext/arto-fi/all/860991.tsv
http://www.yso.fi/onto/yso/p17746 not a skos:Concept
../fulltext/arto-fi/all/861001.tsv
../fulltext/arto-fi/all/862024.tsv
../fulltext/arto-fi/all/862286.tsv
../fulltext/arto-fi/all/862425.tsv
../fulltext/arto-fi/all/862426.tsv
http://www.yso.fi/onto/yso/p9027 not 

../fulltext/arto-fi/all/885828.tsv
../fulltext/arto-fi/all/885831.tsv
../fulltext/arto-fi/all/885836.tsv
../fulltext/arto-fi/all/885861.tsv
../fulltext/arto-fi/all/885862.tsv
http://www.yso.fi/onto/yso/p13389 not a skos:Concept
http://www.yso.fi/onto/yso/p7865 not a skos:Concept
../fulltext/arto-fi/all/885865.tsv
../fulltext/arto-fi/all/885867.tsv
../fulltext/arto-fi/all/885868.tsv
http://www.yso.fi/onto/yso/p21339 not a skos:Concept
../fulltext/arto-fi/all/885870.tsv
http://www.yso.fi/onto/yso/p8053 not a skos:Concept
../fulltext/arto-fi/all/885871.tsv
http://www.yso.fi/onto/yso/p10936 not a skos:Concept
../fulltext/arto-fi/all/885872.tsv
../fulltext/arto-fi/all/885875.tsv
../fulltext/arto-fi/all/885955.tsv
../fulltext/arto-fi/all/885961.tsv
http://www.yso.fi/onto/yso/p15883 not a skos:Concept
../fulltext/arto-fi/all/887348.tsv
http://www.yso.fi/onto/yso/p10636 not a skos:Concept
http://www.yso.fi/onto/yso/p9150 not a skos:Concept
../fulltext/arto-fi/all/887356.tsv
../fulltext/arto-fi

../fulltext/arto-fi/all/960317.tsv
../fulltext/arto-fi/all/960326.tsv
http://www.yso.fi/onto/yso/p8297 not a skos:Concept
http://www.yso.fi/onto/yso/p23244 not a skos:Concept
../fulltext/arto-fi/all/960426.tsv
../fulltext/arto-fi/all/960880.tsv
../fulltext/arto-fi/all/960887.tsv
../fulltext/arto-fi/all/960904.tsv
../fulltext/arto-fi/all/961376.tsv
../fulltext/arto-fi/all/961394.tsv
../fulltext/arto-fi/all/961401.tsv
../fulltext/arto-fi/all/961429.tsv
../fulltext/arto-fi/all/961434.tsv
../fulltext/arto-fi/all/961447.tsv
../fulltext/arto-fi/all/961459.tsv
../fulltext/arto-fi/all/961462.tsv
../fulltext/arto-fi/all/961533.tsv
../fulltext/arto-fi/all/961591.tsv
../fulltext/arto-fi/all/961601.tsv
http://www.yso.fi/onto/yso/p3875 not a skos:Concept
../fulltext/arto-fi/all/961648.tsv
../fulltext/arto-fi/all/961879.tsv
../fulltext/arto-fi/all/961940.tsv
http://www.yso.fi/onto/yso/p14562 not a skos:Concept
../fulltext/arto-fi/all/962483.tsv
http://www.yso.fi/onto/yso/p6597 not a skos:Concept
../

../fulltext/arto-fi/all/996950.tsv
../fulltext/arto-fi/all/996957.tsv
../fulltext/arto-fi/all/996961.tsv
../fulltext/arto-fi/all/998080.tsv
http://www.yso.fi/onto/yso/p18904 not a skos:Concept
../fulltext/arto-fi/all/998095.tsv
../fulltext/arto-fi/all/998102.tsv
../fulltext/arto-fi/all/998152.tsv
../fulltext/arto-fi/all/998165.tsv
http://www.yso.fi/onto/yso/p8493 not a skos:Concept
../fulltext/arto-fi/all/998258.tsv
../fulltext/arto-fi/all/998261.tsv
../fulltext/arto-fi/all/998275.tsv
../fulltext/arto-fi/all/998327.tsv
http://www.yso.fi/onto/yso/p4565 not a skos:Concept
../fulltext/arto-fi/all/998339.tsv
http://www.yso.fi/onto/yso/p10457 not a skos:Concept
../fulltext/arto-fi/all/998345.tsv
../fulltext/arto-fi/all/998397.tsv
../fulltext/arto-fi/all/998428.tsv
../fulltext/arto-fi/all/998561.tsv
../fulltext/arto-fi/all/998692.tsv
../fulltext/arto-fi/all/998705.tsv
../fulltext/arto-fi/all/998719.tsv
../fulltext/arto-fi/all/998725.tsv
http://www.yso.fi/onto/yso/p4398 not a skos:Concept
../

In [ ]:
# This function can be used to update files in the short text corpus format (single TSV file)

def read_short_tsv(fn):
    with open(fn) as tsvf:
        for line in tsvf:
            line = line.strip()
            if '\t' not in line:
                continue
            text, uris = line.split('\t', 1)
            uris = [uri[1:-1] for uri in uris.split()]
            yield (text, uris)

with open('yso-finna-new.tsv', 'w') as newf:
    for text, uris in read_short_tsv('yso-finna.tsv'):
        newuris = []
        for uri in uris:
            for newuri, label in replace_concept((uri, '')):
                if check_concept(newuri):
                    newuris.append(newuri)
        if newuris:
            print("{}\t{}".format(text, '\t'.join(("<{}>".format(uri) for uri in newuris))), file=newf)
